In [12]:
from _util import *
import pandas as pd
import random
import srsly
random.seed(42)
from tqdm import tqdm

In [13]:
def get_text (ad):
    return " ".join([ad["title_parsed"].split(" bei ")[0], ad["description"]])
    
def empty_list(ad):
    return []

In [14]:
def get_number_esco_id(id, length):
    id_short = id[:length]
    return id_short

In [15]:
jobadsdf = pd.DataFrame(load_json("../00_data/jobads_for_classifiertraining.json"))

In [16]:
jobadsdf_copy = jobadsdf.copy()
jobadsdf_copy["text"] = jobadsdf_copy.apply(get_text, axis=1)
jobadsdf_copy["label"] = jobadsdf_copy.apply(empty_list, axis=1)
jobadsdf_copy["esco_filtered"] = jobadsdf_copy["esco_id"].apply(get_number_esco_id, args=[4])
grouped_df = jobadsdf_copy.groupby("esco_filtered").sum().reset_index()
escoids = list(grouped_df["esco_filtered"])
#print(len(escoids))
random_choices = []
for id in tqdm(escoids):
    filtered_df = jobadsdf_copy[jobadsdf_copy["esco_filtered"] == id]
    if len(filtered_df) <= 5:
        random_choices += filtered_df[["text","label"]].to_dict("records")
    else:
        random_choices += random.sample(filtered_df[["text","label"]].to_dict("records"), 5)

100%|██████████| 256/256 [00:02<00:00, 108.10it/s]


In [17]:
srsly.write_jsonl("jsonl_annot_raw.jsonl", random_choices)

# Further process

1. The jsonl file was further processed with [doccano](https://github.com/doccano/doccano)
2. A project for NER was created
3. The relevant paragraphs were marked as "relevant"